In [ ]:
# 추출한 raw ppg r,g,b 신호에서 산소포화도 값 추출하는 방법
# rppg 신호 ---(계산)--> S 신호 ---(선형회귀)--> rspo2 신호
# 비접촉식으로 추출한 산소포화도를 remote spo2 = rspo2로 지정
# 센서로 측정한 spo2는 contact spo2 = cspo2로 지정

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from scipy import signal
from scipy.signal import find_peaks
import pandas as pd
import csv

from sklearn.linear_model import LinearRegression

In [ ]:
def temporal_bandpass_filter(data, fs, minHz, maxHz):
    try:
        nyq = fs / 2
        coef_vector = signal.butter(5, [minHz / nyq, maxHz / nyq], 'bandpass')
        return signal.filtfilt(*coef_vector, data)
    except ValueError:
        return data

def temporal_lowpass_filter(data, fs, minHz):
    try:
        nyq = fs / 2
        coef_vector = signal.butter(5, minHz / nyq, 'lowpass')
        return signal.filtfilt(*coef_vector, data)
    except ValueError:
        return data

## 1.S값 추출

In [ ]:
num=8

fs = 30 #framerate

window_time = 10 #10초씩 분석
window_size = window_time * fs

f = open('E:/project/spo2/code/data/rppg/rppg_rgb%d.csv'%num, 'r').readlines()

r_raw = [float(x) for x in f[0].split(',')]
g_raw = [float(x) for x in f[1].split(',')]
b_raw = [float(x) for x in f[2].split(',')]
t_raw = [x for x in f[3].split(',')] #time
#cspo2는 시간이 '시:분:초' 형식인데 rppg는 시간이 '시-분-초' 형식으로 저장되어서 이를 :로 바꿔줌
t_raw = [i.replace("-",":").strip("\n") for i in t_raw]

#rppg 신호의 앞뒤를 자르고 싶으면 여기서 조정
#보통 실험할 때 시작과 끝에는 노이즈가 많이 포함될 수 있어서 몇초씩 제거하고 하는 경우가 많음
#여기서는 앞의 10초 제거
r_crop = r_raw[10*fs:]
g_crop = g_raw[10*fs:]
b_crop = b_raw[10*fs:]
cropped_t = t_raw[10*fs:]

# RGB to YCgCr color space
r_crop = np.array(r_crop)/255
g_crop = np.array(g_crop)/255
b_crop = np.array(b_crop)/255

y = ((65.481*r_crop) + (128.553*g_crop) + (24.966*b_crop))+16
cropped_cg = ((-81.085*r_crop) + (112*g_crop) + (-30.915*b_crop))+128
cropped_cr = ((112*r_crop) + (-93.786*g_crop) + (-18.214*b_crop))+128
#cbcr 할때는 이렇게 cb 구하면 됨
# cropped_cb =((-37.797*r_crop) + (-74.203*g_crop) + (112*b_crop))+128

# bandpass filter
#최소 심박수 42bpm(0.7Hz)와 최대 심박수 180bpm(3Hz)로 심장박동 관련 신호만 추출
cr_pass = temporal_bandpass_filter(cropped_cr, fs,0.7, 3)
cg_pass = temporal_bandpass_filter(cropped_cg, fs, 0.7, 3)

#ac 구하기
step = len(cr_pass)-window_size+1
ac_cr = []
ac_cg = []
for i in range(step):
    cr_p = cr_pass[i:i+window_size]
    cr_v = -cr_pass[i:i+window_size] #valley를 검출하기 위해서 신호 반전시킴
    cg_p = cg_pass[i:i+window_size]
    cg_v = -cg_pass[i:i+window_size]

    cr_peaks, _ = find_peaks(cr_p, distance=15) #peak 검출
    cr_valleys, _ = find_peaks(cr_v, distance=15) #valley 검출
    cg_peaks, _ = find_peaks(cg_p, distance=15) #peak 검출
    cg_valleys, _ = find_peaks(cg_v, distance=15) #valley 검출

    #peak와 valley의 개수가 같지 않을 수 있어서 최소 개수 지정
    cr_length = len(cr_peaks) if len(cr_peaks)<len(cr_valleys) else len(cr_valleys)
    cg_length = len(cg_peaks) if len(cg_peaks)<len(cg_valleys) else len(cg_valleys)

    cr_peak2valley =[]
    cg_peak2valley =[]

    #|valley 나누기 peak|
    for r in range(cr_length):
        ampl = np.abs(cr_p[cr_valleys[r]]/cr_p[cr_peaks[r]])
        cr_peak2valley.append(ampl)

    for g in range(cg_length):
        ampl = np.abs(cg_p[cg_valleys[g]]/cg_p[cg_peaks[g]])
        cg_peak2valley.append(ampl)

    #중앙값 추출
    #이는 노이즈를 막기 위해 중앙값 선택
    ac_cr.append(np.median(cr_peak2valley))
    ac_cg.append(np.median(cg_peak2valley))

# log
cr_log = np.log(np.array(ac_cr)+1)
cg_log = np.log(np.array(ac_cg)+1)

# 추세만 확인하기 위해 lowpass filtering 수행
cr_cv = temporal_lowpass_filter(np.array(cr_log), fs, 0.05)
cg_cv = temporal_lowpass_filter(np.array(cg_log), fs, 0.05)

#ratio 구하기
ratio = cr_cv/cg_cv

#1초 단위로 합치기
#cspo2가 1초씩 측정되는데 rspo2는 1초에도 값이 여러개이기 때문에 이 과정을 거침
#S가 1초가 한번씩이도록 평균값으로 지정
time = cropped_t[int(window_size):-1]
now_t = time[0]
ratio_sum = 0
cnt = 0
pred_spo2 = [] #최종 예측 rspo2
pred_t = [] #최종 rspo2 time
for i in range(len(time)):
    if now_t==time[i]:
        ratio_sum+= ratio[i]
        cnt+=1
    else:
        pred_spo2.append(ratio_sum/cnt)
        pred_t.append(now_t)

        now_t = time[i]
        ratio_sum = ratio[i]
        cnt=1
    if i == (len(time)-1):
        pred_spo2.append(ratio_sum/cnt)
        pred_t.append(now_t)

#cSpO2 불러오기
df = pd.read_csv('E:/project/spo2/code/data/cppg/cppg/%d.csv'%num)
c_spo2 = df["SPO2"]
c_time = df["TIME"]
c_spo2 = np.array(c_spo2)
c_time = np.array(c_time)
c_t = [i.lstrip() for i in c_time]

#cSpO2와 S 시간축 맞추기
for i in range(len(c_t)):
    if(pred_t[0]==c_t[i]):
        start_t_index = i
    if(pred_t[-1]==c_t[i]):
        end_t_index = i
        break

label_t = c_t[start_t_index:end_t_index+1]
label_spo2 = c_spo2[start_t_index:end_t_index+1]

## 2.S 값과 cspo2 값 저장

In [ ]:
file_num = 20
with open('C:/Users/02097/Desktop/spo2/data/pred/log_%d.csv'%file_num, 'w', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(pred_spo2)
    wr.writerow(pred_t)
    f.close()

with open('C:/Users/02097/Desktop/spo2/data/label/log_%d.csv'%file_num, 'w', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(label_spo2)
    wr.writerow(label_t)
    f.close()

## 3. S와 cspo2의 선형회귀(이건 처음에만)

In [ ]:
#데이터 불러오기(S값과 cspo2값)
cgcr_pred = []
label = []
for i in range(1, 6):
    file_num = i
    label_path = 'E:/project/spo2/code/data/pred/log_%d.csv'%file_num
    label_file = open(label_path,'r').readlines()
    for x in label_file[0].split(','):
        label.append(float(x))

    pred_path = 'E:/project/spo2/code/data/label/log_%d.csv'%file_num
    pred_file = open(pred_path,'r').readlines()
    for x in pred_file[0].split(','):
        cgcr_pred.append(float(x))

#선형회귀
pred_ = np.array(cgcr_pred).reshape(-1,1)

model = LinearRegression()
model.fit(pred_, label)
model.intercept_ , model.coef_

## 4. 선형회귀 결과

In [ ]:
cgcr_pred_ = np.array(cgcr_pred).reshape(-1,1)
cgcr_spo2 = 79.1914 + cgcr_pred_* 11.8805
#cbcr_spo2 = 7.4032 * cbcr_pred_ +87.9765 #cbcr로 했을 경우 선형회귀 결과


## 5. rspo2와 cspo2 저장

In [ ]:
with open('C:/Users/02097/Desktop/spo2/data/graph_signal/signal.csv', 'w', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(label)
    wr.writerow(cgcr_spo2.flatten())
    f.close()